In [48]:
import numpy as np
import tensorflow as tf


In [49]:
users = ['Nare', 'Narek',   'Aram', 'Arpi']

movies = [ 'Oppenheimer',
     'The Dark Knight', 'Shrek',
    'Inside out', "Barbie", 'Star Wars'
]

features = ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']

num_users = len(users)
num_movies = len(movies)
num_feats = len(features)
num_recommendations = 2


In [50]:
users_movies = tf.constant([
                [5,  7,  9,  0, 0, 6],
                [0,  0, 10,  0, 7, 8],
                [0,  6,  0,  0, 5, 6],
                [10, 4,  0,  10, 0, 8]], dtype=tf.float32)


In [51]:
movies_feats = tf.constant([
                [1, 1, 0, 0, 1],
                [1, 1, 0, 0, 0],
                [0, 0, 1, 1, 0],
                [1, 0, 1, 1, 0],
                [0, 0, 0, 0, 1],
                [1, 0, 0, 0, 1]], dtype=tf.float32)


In [52]:
users_feats = tf.matmul(users_movies, movies_feats)
users_feats


<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[18., 12.,  9.,  9., 11.],
       [ 8.,  0., 10., 10., 15.],
       [12.,  6.,  0.,  0., 11.],
       [32., 14., 10., 10., 18.]], dtype=float32)>

In [53]:
users_feats = users_feats / tf.reduce_sum(users_feats, axis=1, keepdims=True)
users_feats


<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0.30508474, 0.20338982, 0.15254237, 0.15254237, 0.18644068],
       [0.18604651, 0.        , 0.23255815, 0.23255815, 0.3488372 ],
       [0.41379312, 0.20689656, 0.        , 0.        , 0.37931034],
       [0.3809524 , 0.16666667, 0.11904762, 0.11904762, 0.21428572]],
      dtype=float32)>

In [54]:
top_users_features = tf.nn.top_k(users_feats, num_feats)[1]
top_users_features


<tf.Tensor: shape=(4, 5), dtype=int32, numpy=
array([[0, 1, 4, 2, 3],
       [4, 2, 3, 0, 1],
       [0, 4, 1, 2, 3],
       [0, 4, 1, 2, 3]], dtype=int32)>

In [55]:
for i in range(num_users):
    feature_names = [features[int(index)] for index in top_users_features[i]]
    print('{}: {}'.format(users[i], feature_names))


Nare: ['Action', 'Sci-Fi', 'Drama', 'Comedy', 'Cartoon']
Narek: ['Drama', 'Comedy', 'Cartoon', 'Action', 'Sci-Fi']
Aram: ['Action', 'Drama', 'Sci-Fi', 'Comedy', 'Cartoon']
Arpi: ['Action', 'Drama', 'Sci-Fi', 'Comedy', 'Cartoon']


In [56]:
users_ratings =  [tf.map_fn(lambda x: tf.tensordot(users_feats[i], x, axes = 1),
                           tf.cast(movies_feats, tf.float32))
                  for i in range(num_users)]


In [57]:
users_unseen_movies = tf.equal(users_movies, tf.zeros_like(users_movies))
users_unseen_movies


<tf.Tensor: shape=(4, 6), dtype=bool, numpy=
array([[False, False, False,  True,  True, False],
       [ True,  True, False,  True, False, False],
       [ True, False,  True,  True, False, False],
       [False, False,  True, False,  True, False]])>

In [58]:
ignore_matrix = tf.zeros_like(tf.cast(users_movies, tf.float32))
ignore_matrix

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [59]:
users_ratings_new = tf.where(
    users_unseen_movies,
    users_ratings,
    ignore_matrix)

users_ratings_new


<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.6101695 , 0.18644068,
        0.        ],
       [0.53488374, 0.18604651, 0.        , 0.6511628 , 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , 0.41379312, 0.        ,
        0.        ],
       [0.        , 0.        , 0.23809524, 0.        , 0.21428572,
        0.        ]], dtype=float32)>

In [60]:
top_movies = tf.nn.top_k(users_ratings_new, num_recommendations)[1]
top_movies


<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[3, 4],
       [3, 0],
       [0, 3],
       [2, 4]], dtype=int32)>

In [61]:
for i in range(num_users):
    movie_names = [movies[index] for index in top_movies[i]]
    print('{}: {}'.format(users[i], movie_names))


Nare: ['Inside out', 'Barbie']
Narek: ['Inside out', 'Oppenheimer']
Aram: ['Oppenheimer', 'Inside out']
Arpi: ['Shrek', 'Barbie']


In [ ]:
Sources



[Google Course “Recommendation systems”](https://developers.google.com/machine-learning/recommendation/overview/terminology)

[Content-based Recommender System for Movies with Tensorflow](https://www.doczamora.com/content-based-recommender-system-for-movies-with-tensorflow)

[A Hybrid Approach using Collaborative filtering and Content-based Filtering for Recommender
System](https://iopscience.iop.org/article/10.1088/1742-6596/1000/1/012101/pdf#:~:text=A%20hybrid%20approach%20is%20taken,the%20performance%20of%20the%20system.)

[Personalized Movie Recommendation Engine using Content-based Filtering in Python](https://www.relataly.com/content-based-movie-recommender-using-python/4294/)